In [1]:

import mysql.connector
import pandas as pd


In [2]:

# Connect to MySQL
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='sumitx2903',
    password='sumitsolanki2903',  # Replace with your MySQL password
    database='ott_insight_hub'
)
cursor = conn.cursor()


In [7]:

# Read cleaned CSV
csv_path = '../../Data_Extraction_and_Cleaning/Cleaned/Cleaned_CSV/movies_tmdb_cleaned.csv'
df = pd.read_csv(csv_path)

print("Starting data import...")


Starting data import...


In [8]:

# Step 1: Insert all unique genres into dim_genre
print("Inserting genres...")
all_genres = set()
for genres_str in df['Genre(s)']:
    genres = [g.strip() for g in str(genres_str).split(',')]
    all_genres.update(genres)

for genre in all_genres:
    cursor.execute('INSERT IGNORE INTO dim_genre (genre_name) VALUES (%s)', (genre,))

conn.commit()
print(f"Inserted {len(all_genres)} genres")


Inserting genres...
Inserted 17 genres


In [11]:

# Step 2: Insert movies into fact_movies
print("Inserting movies...")
for idx, row in df.iterrows():
    try:
        # Extract year from Release Year string
        release_year_str = str(row['Release Year'])
        # Extract last 4 digits (the actual year)
        release_year = int(release_year_str[-4:])
        
        cursor.execute('''
            INSERT IGNORE INTO fact_movies (movie_name, imdb_rating, num_votes, release_year, runtime, popularity)
            VALUES (%s, %s, %s, %s, %s, %s)
        ''', (
            row['Movie/Show Name'],
            float(row['IMDb Rating']),
            int(row['Number of Votes']),
            release_year,
            int(row['Runtime (optional)']) if pd.notna(row['Runtime (optional)']) else None,
            float(row['Popularity Index (from API if available)']) if pd.notna(row['Popularity Index (from API if available)']) else None
        ))
    except Exception as e:
        print(f"Error inserting movie {row['Movie/Show Name']}: {e}")

conn.commit()
print(f"Inserted {len(df)} movies")


Inserting movies...
Inserted 120 movies


In [12]:

# Step 3: Insert movie-genre relationships into bridge table
print("Creating movie-genre relationships...")
for idx, row in df.iterrows():
    try:
        # Get movie_id
        cursor.execute('SELECT movie_id FROM fact_movies WHERE movie_name = %s', (row['Movie/Show Name'],))
        result = cursor.fetchone()
        if result:
            movie_id = result[0]
            
            # Get genres and insert into bridge table
            genres = [g.strip() for g in str(row['Genre(s)']).split(',')]
            for genre in genres:
                cursor.execute('SELECT genre_id FROM dim_genre WHERE genre_name = %s', (genre,))
                genre_result = cursor.fetchone()
                if genre_result:
                    genre_id = genre_result[0]
                    cursor.execute(
                        'INSERT IGNORE INTO movie_genre_bridge (movie_id, genre_id) VALUES (%s, %s)',
                        (movie_id, genre_id)
                    )
    except Exception as e:
        print(f"Error processing genres for {row['Movie/Show Name']}: {e}")

conn.commit()
print("Created movie-genre relationships")


Creating movie-genre relationships...
Created movie-genre relationships


In [13]:

# Step 4: Insert ratings
print("Inserting ratings...")
for idx, row in df.iterrows():
    try:
        cursor.execute('SELECT movie_id FROM fact_movies WHERE movie_name = %s', (row['Movie/Show Name'],))
        result = cursor.fetchone()
        if result:
            movie_id = result[0]
            cursor.execute('''
                INSERT INTO ratings (movie_id, rating_source, rating_value, rating_count)
                VALUES (%s, %s, %s, %s)
            ''', (movie_id, 'IMDb', float(row['IMDb Rating']), int(row['Number of Votes'])))
    except Exception as e:
        print(f"Error inserting rating for {row['Movie/Show Name']}: {e}")

conn.commit()
cursor.close()
conn.close()
print("✅ Data import completed successfully!")

Inserting ratings...
✅ Data import completed successfully!
